The basic idea of this notebook is generate clusters with K-Means or HDBScan and use the number of the cluster as a new feature. 

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
data_train = pd.read_csv('https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_1_nacho/intento_1/train_no_duplicates_with_word_classes.csv',\
                         sep=',', usecols = ['url_count',
       'hashtag_count', 'mention_count', 'digits_count', 'characters_count',
       'characters_count_clean', 'characters_count_clean_sw', 'word_count',
       'word_count_clean', 'word_count_clean_sw', 'avg_word_len',
       'avg_word_len_clean_sw', 'PRP$', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'MD',
       'VB', 'JJ', 'PRP', 'JJS', 'VBD', 'TO', 'VBG', 'VBN', 'CC', 'CD', 'RB',
       'EX', 'VBZ', 'WP', 'RP', 'JJR', 'WRB', '$', 'WDT', 'NNP', 'RBR', 'RBS',
       'PDT', 'SYM', 'FW', 'UH', 'X', 'WP$', 'target'],\
       dtype = {'PRP$': 'int64', 'NNS': 'int64', 'VBP': 'int64', 'DT': 'int64', 'NN': 'int64', 'IN': 'int64', 'MD': 'int64',
       'VB': 'int64', 'JJ': 'int64', 'PRP': 'int64', 'JJS': 'int64', 'VBD': 'int64', 'TO': 'int64', 'VBG': 'int64', 'VBN': 'int64',\
       'CC': 'int64', 'CD': 'int64', 'RB': 'int64','EX': 'int64', 'VBZ': 'int64', 'WP': 'int64', 'RP': 'int64', 'JJR': 'int64',\
       'WRB': 'int64', '$': 'int64', 'WDT': 'int64', 'NNP': 'int64', 'RBR': 'int64', 'RBS': 'int64','PDT': 'int64', 'SYM': 'int64',\
       'FW': 'int64', 'UH': 'int64', 'X': 'int64', 'WP$': 'int64', 'target': 'int64'})



data_test = pd.read_csv('https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_1_nacho/intento_1/featured_test.csv',sep=',')
data_test = data_test[data_test.columns[1:]]
data_test.columns = ['id', 'keyword', 'location', 'text', 'text_clean', 'text_clean_no_sw',
       'url_count', 'hashtag_count', 'mention_count', 'digits_count',
       'characters_count', 'characters_count_clean',
       'characters_count_clean_sw', 'word_count', 'word_count_clean',
       'word_count_clean_sw', 'avg_word_len', 'avg_word_len_clean_sw',
       'tokenized_text', 'pos_tagged_text', 'pos_tagg_counts', 'RB', 'VBD',
       'DT', 'JJ', 'NN', 'IN', 'VBZ', 'NNS', 'VBP', 'EX', 'VBG', 'VB', 'PRP',
       'CD', 'CC', 'MD', 'RBR', 'WRB', 'WP', 'VBN', 'RP', 'PRP$', '$', 'TO',
       'NNP', 'WDT', 'PDT', 'JJS', 'JJR', 'FW', 'RBS', 'X', 'UH']
id = list(data_test.id)
x_test_numeric = data_test[['url_count', 'hashtag_count', 'mention_count', 'digits_count',
       'characters_count', 'characters_count_clean',
       'characters_count_clean_sw', 'word_count', 'word_count_clean',
       'word_count_clean_sw', 'avg_word_len', 'avg_word_len_clean_sw',
       'RB', 'VBD','DT', 'JJ', 'NN', 'IN', 'VBZ', 'NNS', 'VBP', 'EX', 'VBG', 'VB', 'PRP',
       'CD', 'CC', 'MD', 'RBR', 'WRB', 'WP', 'VBN', 'RP', 'PRP$', '$', 'TO',
       'NNP', 'WDT', 'PDT', 'JJS', 'JJR', 'FW', 'RBS', 'X', 'UH']]

drop = []
for i in list(data_test.columns): 
  if i not in list(data_train.columns): 
    drop.append(i)

data_test.drop(columns= drop, inplace=True)

drop = []
for i in list(data_train.columns): 
  if i not in list(data_test.columns) and i != 'target':
    drop.append(i)

data_train.drop(columns=drop, inplace=True)

X_train, y_train = data_train.iloc[:,1:], data_train['target']
X_test = data_test

X_train.fillna(value=0, inplace = True)
X_test.fillna(value=0, inplace = True)


In [4]:
X_test = X_test[['url_count', 'hashtag_count', 'mention_count', 'digits_count',
       'characters_count', 'characters_count_clean',
       'characters_count_clean_sw', 'word_count', 'word_count_clean',
       'word_count_clean_sw', 'avg_word_len', 'avg_word_len_clean_sw', 'PRP$',
       'NNS', 'VBP', 'DT', 'NN', 'IN', 'MD', 'VB', 'JJ', 'PRP', 'JJS', 'VBD',
       'TO', 'VBG', 'VBN', 'CC', 'CD', 'RB', 'EX', 'VBZ', 'WP', 'RP', 'JJR',
       'WRB', '$', 'WDT', 'NNP', 'RBR', 'RBS', 'PDT', 'FW', 'UH', 'X']]

# Using HDBScan

In [5]:
!pip install hdbscan

     |████████████████████████████████| 4.7MB 4.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2305918 sha256=8af009671b55f14f0663578b821cbac0ec03742c6c607024c9599fb2181a5b6e
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan


In [15]:
X_train['istrain'] = 1

In [16]:
X_test['istrain'] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
appended = X_train.append(X_test)

In [6]:
import hdbscan

In [7]:
clusterer = hdbscan.HDBSCAN()

In [21]:
clusterer.fit(appended)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='euclidean', min_cluster_size=5, min_samples=None, p=None,
        prediction_data=False)

In [22]:
cluster_labels_hdbscan_appended = clusterer.labels_

In [23]:
appended['cluster_labels_hdbscan'] = cluster_labels_hdbscan_appended

In [26]:
X_train = appended[appended['istrain']==1]
X_test = appended[appended['istrain']==0]
X_train = X_train.drop(columns='istrain')
X_test = X_test.drop(columns='istrain')

In [28]:
X_train.to_csv('X_train.csv', sep=',', index=False)
X_test.to_csv('X_test.csv', sep=',', index=False)